In [ ]:
import pandas as pd
import re
from transformers import pipeline
from tqdm import tqdm

# Pour afficher la progression
tqdm.pandas()

# Chargement du dataset
data = pd.read_csv("reviews_0-250.csv")

# Afficher les 5 premières lignes pour vérifier
print(data.head())

<ipython-input-1-606f2897d513>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("reviews_0-250.csv")


   Unnamed: 0    author_id  rating  is_recommended  helpfulness  \
0           0   1741593524       5             1.0          1.0   
1           1  31423088263       1             0.0          NaN   
2           2   5061282401       5             1.0          NaN   
3           3   6083038851       5             1.0          NaN   
4           4  47056667835       5             1.0          NaN   

   total_feedback_count  total_neg_feedback_count  total_pos_feedback_count  \
0                     2                         0                         2   
1                     0                         0                         0   
2                     0                         0                         0   
3                     0                         0                         0   
4                     0                         0                         0   

  submission_time                                        review_text  \
0      2023-02-01  I use this with the Nudestix “C

In [ ]:
# Nettoyage du texte des reviews
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Minuscule
        text = re.sub(r'[^\w\s]', '', text)  # Supprime ponctuation
        text = re.sub(r'\d+', '', text)  # Supprime chiffres
        text = text.strip()
    return text



In [ ]:
# Application du nettoyage
data['cleaned_review'] = data['review_text'].apply(clean_text)

# Supprimer les lignes sans texte nettoyé
data = data.dropna(subset=['cleaned_review'])


In [ ]:

# Charger le pipeline de sentiment analysis (avec PyTorch)
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model_name,
    tokenizer=model_name,
    framework="pt"  # PyTorch
)

# Conversion des labels vers scores
def sentiment_to_score(sentiment_label):
    if sentiment_label == "LABEL_2":  # positive
        return 1
    elif sentiment_label == "LABEL_1":  # neutral
        return 0
    else:  # LABEL_0 = negative
        return -1

# Analyse d'une seule review (avec gestion d'erreur)
def analyze_review_sentiment(text):
    try:
        if not isinstance(text, str) or text.strip() == "":
            return {'sentiment_label': 'LABEL_1', 'sentiment_score': 0}  # neutre par défaut
        result = sentiment_pipeline(text[:512])[0]  # tronque à 512 tokens max
        return {
            'sentiment_label': result['label'],
            'sentiment_score': sentiment_to_score(result['label'])
        }
    except Exception as e:
        print(f"Erreur sur le texte : {text[:30]}... | {e}")
        return {
            'sentiment_label': 'LABEL_1',
            'sentiment_score': 0
        }



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
results = data['cleaned_review'].progress_apply(analyze_review_sentiment)
results_df = pd.DataFrame(results.tolist())
data = pd.concat([data.reset_index(drop=True), results_df], axis=1)
data.to_csv("reviews_with_sentiment.csv", index=False)
from google.colab import files
files.download("reviews_with_sentiment.csv")

 68%|██████▊   | 406351/601131 [1:05:04<33:37, 96.53it/s]

Erreur sur le texte : ɪ ʀᴇᴄᴇɪᴠᴇᴅ ᴛʜᴇꜱᴇ ᴘʀᴏᴅᴜᴄᴛꜱ ᴄᴏᴍᴘ... | The expanded size of the tensor (854) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 854].  Tensor sizes: [1, 514]


100%|██████████| 601131/601131 [1:35:38<00:00, 104.75it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Evaluation du modèle en utilisant rating
def rating_to_sentiment(rating):
    if rating >= 4:
        return 1     # positive
    elif rating == 3:
        return 0     # neutral
    else:
        return -1    # negative

#  Appliquer cette fonction à ta colonne 'rating'
data['rating_sentiment'] = data['rating'].apply(rating_to_sentiment)

#  Comparer au 'sentiment_score' prédéfini
correct = data['rating_sentiment'] == data['sentiment_score']

# 📊 Calcul de l'accuracy
accuracy = correct.mean()
print(f" Accuracy du modèle de sentiment par rapport aux ratings : {accuracy:.2%}")



 Accuracy du modèle de sentiment par rapport aux ratings : 86.08%
